# Lesson 4 — Add New Blog Posts with a Form

## 🎯 What we'll build
In this lesson, we will:
1. Learn how **JSON files store data**
2. Create a **form page** to add new blog posts
3. Write Python code to **save new posts** to `blogs.json`
4. See our new posts appear on the home page!

By the end, you'll have a **complete blog system** where you can:
- ✅ View all blog posts (Lesson 3)
- ✅ Read individual posts (Lesson 3)
- ✅ **Add new posts using a form** (Lesson 4)


## 📚 Step 1 — Understanding JSON Structure

Let's review what our `blogs.json` file looks like:

```json
[
  {
    "id": 1,
    "slug": "my-first-day-with-flask",
    "title": "My First Day with Flask",
    "author": "Aisha",
    "date": "2025-11-08",
    "image_url": "https://picsum.photos/id/1035/1200/600",
    "paragraphs": [
      "Today I started learning Flask...",
      "I learned how to create a route..."
    ]
  }
]
```

**Key Points:**
- Each blog post is an **object** with fields like `id`, `title`, `author`
- The `slug` is a URL-friendly version of the title
- The `paragraphs` field is a **list** of text paragraphs
- All posts are stored in a **list** (the `[ ]` brackets)


## 🛠️ Step 2 — Add Helper Functions to `app.py`

Before we create the form, we need helper functions.

**Add these functions to your `app.py` after `load_blogs()`:**


In [ ]:
from datetime import datetime
import re

def create_slug(title):
    """Convert a title into a URL-friendly slug"""
    slug = title.lower()
    slug = re.sub(r'[^a-z0-9\s-]', '', slug)
    slug = re.sub(r'\s+', '-', slug)
    return slug

def get_next_id():
    """Get the next available ID for a new blog post"""
    blogs = load_blogs()
    if not blogs:
        return 1
    max_id = max(blog.get('id', 0) for blog in blogs)
    return max_id + 1

def save_blogs(blogs):
    """Save the blogs list back to the JSON file"""
    with open(DATA_FILE, 'w', encoding='utf-8') as f:
        json.dump(blogs, f, indent=2, ensure_ascii=False)

### 🤔 What do these functions do?

1. **`create_slug(title)`** - Converts "My Amazing Post" → "my-amazing-post"
2. **`get_next_id()`** - Finds highest ID and adds 1
3. **`save_blogs(blogs)`** - Writes blogs back to `blogs.json`


## 📝 Step 3 — Create the "Add Post" Form Route

**Add this route to your `app.py`:**


In [ ]:
from flask import request, redirect

@app.route('/add-post', methods=['GET', 'POST'])
def add_post():
    if request.method == 'POST':
        # Get data from the form
        title = request.form.get('title', '').strip()
        author = request.form.get('author', '').strip()
        image_url = request.form.get('image_url', '').strip()
        paragraph1 = request.form.get('paragraph1', '').strip()
        paragraph2 = request.form.get('paragraph2', '').strip()
        paragraph3 = request.form.get('paragraph3', '').strip()
        
        # Validate required fields
        if not title or not author:
            return "Error: Title and Author are required!", 400
        
        # Create paragraphs list
        paragraphs = [p for p in [paragraph1, paragraph2, paragraph3] if p]
        
        # Create the new blog post
        new_post = {
            'id': get_next_id(),
            'slug': create_slug(title),
            'title': title,
            'author': author,
            'date': datetime.now().strftime('%Y-%m-%d'),
            'image_url': image_url or 'https://picsum.photos/id/1015/1200/600',
            'paragraphs': paragraphs
        }
        
        # Load, add, and save
        blogs = load_blogs()
        blogs.append(new_post)
        save_blogs(blogs)
        
        # Redirect to home page
        return redirect(url_for('home'))
    
    # Show the form
    return render_template('add_post.html')

## 🎨 Step 4 — Create the Form Template

**Create a new file `templates/add_post.html`:**


In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="utf-8" />
  <meta name="viewport" content="width=device-width,initial-scale=1" />
  <title>Add New Post</title>
  <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>
<body>
  <header class="site-header">
    <h1>✍️ Add New Blog Post</h1>
    <p>Share your thoughts!</p>
  </header>

  <main class="form-container">
    <form method="POST" action="{{ url_for('add_post') }}" class="blog-form">
      
      <div class="form-group">
        <label for="title">Post Title *</label>
        <input type="text" id="title" name="title" required 
               placeholder="My Amazing Blog Post">
      </div>

      <div class="form-group">
        <label for="author">Your Name *</label>
        <input type="text" id="author" name="author" required 
               placeholder="Your name">
      </div>

      <div class="form-group">
        <label for="image_url">Image URL (optional)</label>
        <input type="url" id="image_url" name="image_url" 
               placeholder="https://picsum.photos/id/1015/1200/600">
        <small>Leave empty for default image</small>
      </div>

      <div class="form-group">
        <label for="paragraph1">Paragraph 1</label>
        <textarea id="paragraph1" name="paragraph1" rows="3" 
                  placeholder="Write your first paragraph..."></textarea>
      </div>

      <div class="form-group">
        <label for="paragraph2">Paragraph 2 (optional)</label>
        <textarea id="paragraph2" name="paragraph2" rows="3" 
                  placeholder="Write your second paragraph..."></textarea>
      </div>

      <div class="form-group">
        <label for="paragraph3">Paragraph 3 (optional)</label>
        <textarea id="paragraph3" name="paragraph3" rows="3" 
                  placeholder="Write your third paragraph..."></textarea>
      </div>

      <div class="form-actions">
        <button type="submit" class="btn btn-primary">📝 Publish Post</button>
        <a href="{{ url_for('home') }}" class="btn btn-secondary">Cancel</a>
      </div>
    </form>
  </main>
</body>
</html>

## 🎨 Step 5 — Add Form Styles to CSS

**Add these styles to `static/style.css`:**


In [ ]:
/* Form Styles */
.form-container {
  max-width: 700px;
  margin: 20px auto;
  padding: 0 12px;
}

.blog-form {
  background: #fff;
  padding: 24px;
  border-radius: 14px;
  box-shadow: 0 4px 14px rgba(0, 0, 0, 0.08);
}

.form-group {
  margin-bottom: 20px;
}

.form-group label {
  display: block;
  margin-bottom: 6px;
  font-weight: 600;
  color: #333;
}

.form-group input,
.form-group textarea {
  width: 100%;
  padding: 10px 12px;
  border: 2px solid #e0e0e0;
  border-radius: 8px;
  font-size: 15px;
  font-family: system-ui, Arial, sans-serif;
}

.form-group input:focus,
.form-group textarea:focus {
  outline: none;
  border-color: #2f80ed;
}

.form-group small {
  display: block;
  margin-top: 4px;
  color: #666;
  font-size: 13px;
}

.form-actions {
  display: flex;
  gap: 12px;
  margin-top: 24px;
}

.btn-primary {
  background: #22c55e;
  color: #fff;
  border: none;
  padding: 12px 24px;
  border-radius: 10px;
  font-size: 16px;
  cursor: pointer;
}

.btn-primary:hover {
  filter: brightness(0.95);
}

.btn-secondary {
  background: #e0e0e0;
  color: #333;
  padding: 12px 24px;
  border-radius: 10px;
  text-decoration: none;
  display: inline-block;
}

.btn-secondary:hover {
  filter: brightness(0.95);
}

## 🔗 Step 6 — Add Link to Form on Home Page

**Update `templates/home.html` header:**


In [ ]:
<header class="site-header">
  <h1>🌙 Nozolan Kids Blog</h1>
  <p>Learn Python • Build Character</p>
  <a href="{{ url_for('add_post') }}" class="btn" style="margin-top: 12px;">✍️ Write New Post</a>
</header>

## 🎉 Step 7 — Complete `app.py` Code

Here's your **complete `app.py`**:


In [ ]:
from flask import Flask, jsonify, render_template, abort, url_for, request, redirect
import json
from pathlib import Path
from datetime import datetime
import re

app = Flask(__name__)
DATA_FILE = Path(__file__).parent / "blogs.json"

def load_blogs():
    if not DATA_FILE.exists():
        return []
    with open(DATA_FILE, encoding="utf-8") as f:
        return json.load(f)

def save_blogs(blogs):
    with open(DATA_FILE, 'w', encoding='utf-8') as f:
        json.dump(blogs, f, indent=2, ensure_ascii=False)

def create_slug(title):
    slug = title.lower()
    slug = re.sub(r'[^a-z0-9\s-]', '', slug)
    return re.sub(r'\s+', '-', slug)

def get_next_id():
    blogs = load_blogs()
    if not blogs:
        return 1
    return max(blog.get('id', 0) for blog in blogs) + 1

@app.route("/api/blogs")
def api_blogs():
    return jsonify(load_blogs())

@app.route("/")
def home():
    blogs = load_blogs()
    try:
        blogs = sorted(blogs, key=lambda b: b.get("date", ""), reverse=True)
    except:
        pass
    return render_template("home.html", blogs=blogs)

@app.route("/post/<slug>") 
def post_detail(slug):
    blogs = load_blogs()
    for b in blogs:
        if b.get("slug") == slug:
            return render_template("post.html", blog=b)
    abort(404)

@app.route('/add-post', methods=['GET', 'POST'])
def add_post():
    if request.method == 'POST':
        title = request.form.get('title', '').strip()
        author = request.form.get('author', '').strip()
        image_url = request.form.get('image_url', '').strip()
        paragraph1 = request.form.get('paragraph1', '').strip()
        paragraph2 = request.form.get('paragraph2', '').strip()
        paragraph3 = request.form.get('paragraph3', '').strip()
        
        if not title or not author:
            return "Error: Title and Author are required!", 400
        
        paragraphs = [p for p in [paragraph1, paragraph2, paragraph3] if p]
        
        new_post = {
            'id': get_next_id(),
            'slug': create_slug(title),
            'title': title,
            'author': author,
            'date': datetime.now().strftime('%Y-%m-%d'),
            'image_url': image_url or 'https://picsum.photos/id/1015/1200/600',
            'paragraphs': paragraphs
        }
        
        blogs = load_blogs()
        blogs.append(new_post)
        save_blogs(blogs)
        
        return redirect(url_for('home'))
    
    return render_template('add_post.html')

## 🚀 Step 8 — Test Your Blog!

1. **Upload all files to PythonAnywhere**
2. **Reload your web app**
3. **Visit your site** and click "✍️ Write New Post"
4. **Fill out the form** and publish!
5. **See your post appear** on the home page! 🎉


## 🤓 How Does It Work?

When you submit the form:
1. Flask receives the data through `request.form`
2. Python creates a new blog object with auto-generated ID, slug, and date
3. Python loads current blogs from `blogs.json`
4. Python adds your new post to the list
5. Python saves everything back to `blogs.json`
6. Flask redirects you to see your new post! 🎉


## 🏆 Challenges

1. **Add 5 paragraphs** instead of 3
2. **Add a category field** (Coding, Science, etc.)
3. **Show success message** after publishing
4. **Add ability to edit posts** (advanced!)
5. **Validate image URLs** before saving


## 📝 Recap

You learned:
- ✅ How JSON files store data
- ✅ How to create HTML forms
- ✅ How to handle form submissions in Flask
- ✅ How to read and write JSON files
- ✅ How to auto-generate IDs, slugs, and dates

**You now have a complete blog system!** 🌟
